In [1]:
%load_ext autoreload

In [2]:
from lovely_tensors.patch import monkey_patch

monkey_patch()
import torch
from transformers import GPT2Tokenizer
import wandb
from tqdm import tqdm

import json
import os
from pathlib import Path

In [3]:
# from datasets import load_dataset

# dataset = load_dataset("roneneldan/TinyStories")

In [31]:
# Pre-process the tiny stories dataset

# TS_PATH = "./ts/"
# TS_PATH = Path(TS_PATH)


# gpt35_stories = []
# gpt4_stories = []

import unidecode

# for file in tqdm(list(sorted(os.listdir(TS_PATH)))):
#     if file.endswith(".json"):
#         with open(TS_PATH / file, "r") as f:
#             data = json.load(f)
#             for d in data:
#                 story = d["story"]
#                 if not all(ord(c) < 128 for c in story):
#                     story = unidecode.unidecode(story)

#                 if d["source"] == "GPT-3.5":
#                     gpt35_stories.append(story)
#                 elif d["source"] == "GPT-4":
#                     gpt4_stories.append(story)

# with open("gpt35_stories.txt", "w") as f:
#     f.write("\n".join(gpt35_stories))

# with open("gpt4_stories.txt", "w") as f:
#     f.write("\n".join(gpt4_stories))

with open("gpt35_stories.txt", "r") as f:
    gpt35_stories = f.read().split("\n")

with open("gpt4_stories.txt", "r") as f:
    gpt4_stories = f.read().split("\n")

In [32]:
data = "\n".join(gpt35_stories + gpt4_stories)
del gpt35_stories
del gpt4_stories

In [6]:
chars = sorted(list(set(data)))

# create a mapping from characters to integers
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}


def encode(s):
    return [stoi[c] for c in s]  # encoder: take a string, output a list of integers


def decode(l):
    return "".join(
        [itos[i] for i in l]
    )  # decoder: take a list of integers, output a string

In [7]:
"".join(chars), len(chars)

('\t\n !"#$%&\'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]_`abcdefghijklmnopqrstuvwxyz{|}~',
 96)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [33]:
# encoded_data = encode(data)
train_data = data[: int(len(data)) - 200_000]
val_data = data[int(len(data)) - 200_000 :]
del data

# train_data = torch.tensor(data)
# val_data = torch.tensor(val_data)

In [23]:
def get_item(data, ctx):
    # i = random.randint(0, len(data) - ctx - 1)
    i = 0
    while i + ctx < len(data):
        src = data[i : i + ctx]
        dst = data[i + 1 : i + ctx + 1]
        yield torch.tensor(encode(src)), torch.tensor(encode(dst))
        i += ctx


def get_epoch(data, ctx_len, batch_size, shuffle=True):
    """Yields a tuple of tensors of shape (batch_size, ctx).
    X, shape=B C
    y, shape=B C
    """

    items = get_item(data, ctx_len)

    try:
        while True:
            X, y = zip(*[next(items) for _ in range(batch_size)])
            yield torch.stack(X), torch.stack(y)
    except StopIteration:
        pass

In [16]:
import random


def get_random_item(data, ctx):
    i = random.randint(0, len(data) - ctx - 1)

    src = data[i : i + ctx]
    dst = data[i + 1 : i + ctx + 1]

    return torch.tensor(encode(src)), torch.tensor(encode(dst))


def get_batch(data, ctx_len, batch_size, shuffle=True):
    """Yields a tuple of tensors of shape (batch_size, ctx).
    X, shape=B C
    y, shape=B C
    """

    batch = [get_random_item(data, ctx_len) for _ in range(batch_size)]
    X, y = zip(*batch)

    return torch.stack(X), torch.stack(y)


get_batch(train_data[:100], ctx_len=5, batch_size=2)

(tensor[2, 5] i64 n=10 x∈[2, 90] μ=61.400 σ=32.215 [[85, 70, 69, 69, 90], [2, 84, 73, 70, 2]],
 tensor[2, 5] i64 n=10 x∈[2, 90] μ=60.600 σ=31.659 [[70, 69, 69, 90, 2], [84, 73, 70, 2, 77]])

44.2 ms ± 932 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [17]:
def model(params, input_ids, vocab_size):
    """This model takes in a sequence and predicts 1 token"""

    one_hot_inputs = torch.nn.functional.one_hot(input_ids, num_classes=vocab_size)
    one_hot_inputs = one_hot_inputs.float()

    embeddings = one_hot_inputs @ params["embedding"].T  # N, CTX_LEN, EMBEDDING_DIM

    # preds = hidden_states[:, -1, :] # @ params["w"]

    hidden_state = (
        embeddings.view((input_ids.shape[0], -1)) @ params["w1"].T + params["b1"]
    )
    hidden_state = torch.nn.functional.relu(hidden_state)

    hidden_state = (
        hidden_state.view((input_ids.shape[0], -1)) @ params["w2"].T + params["b2"]
    )
    hidden_state = torch.nn.functional.relu(hidden_state)

    preds = hidden_state @ params["embedding"]

    return preds

In [53]:
def generate(model, model_params, ctx_len, encoded_prompt, n_tokens, temperature=0.0, top_k=1):
    """Generate n_tokens after prompt"""
    with torch.no_grad():
        for _ in range(n_tokens):
            # print(encoded_prompt)
            preds = model(
                model_params, encoded_prompt[:, -ctx_len:], vocab_size=len(chars)
            )

            probs = torch.nn.functional.softmax(preds, dim=-1).pow(
                1 / (temperature + 1e-6)
            )

            # Implementing top-k sampling
            top_k_probs, top_k_indices = torch.topk(probs, top_k)
            next_token = torch.multinomial(top_k_probs, num_samples=1)
            next_token = top_k_indices.gather(dim=1, index=next_token)

            encoded_prompt = torch.cat([encoded_prompt, next_token], dim=1)
    return encoded_prompt

In [36]:
print(f"Length of the training data: {len(train_data):,}")

Length of the training data: 3,849,741,212


In [39]:
CTX_LEN = 12

EMBEDDING_DIM = 128

INTERMEDIATE_DIM = EMBEDDING_DIM * 8

BATCH_SIZE = 4096
LR = 0.1

TRAIN_TOKENS = 100_000_000


LOG_INTERVAL = min(
    (min(TRAIN_TOKENS, len(train_data)) // (BATCH_SIZE * CTX_LEN) // 10) + 1, 1000
)
VALIDATION_INTERVAL = LOG_INTERVAL * 2


run = wandb.init(
    # set the wandb project where this run will be logged
    project="my-awesome-project",
    entity="llmnerds",
    config={
        "batch_size": BATCH_SIZE,
        "ctx": CTX_LEN,
    },
)

model_params = {
    "embedding": torch.randn((EMBEDDING_DIM, len(chars)), device=device),
    "w1": torch.randn((INTERMEDIATE_DIM, EMBEDDING_DIM * CTX_LEN), device=device),
    "b1": torch.randn((INTERMEDIATE_DIM,), device=device),
    "w2": torch.randn((EMBEDDING_DIM, INTERMEDIATE_DIM), device=device),
    "b2": torch.randn((EMBEDDING_DIM,), device=device),
    # "classifier": torch.randn(
    #     (INTERMEDIATE_DIM, len(chars)), device=device, requires_grad=True
    # ),
}

# # glorot init
for p in model_params.values():
    if len(p.shape) == 2:
        torch.nn.init.kaiming_normal_(p)
    p.requires_grad = True


i = 1
total_loss = 0
val_total_loss = 0

token_count = 0
optim = torch.optim.Adam(model_params.values(), lr=1e-3)

# batch_gen = get_batch(train_data, ctx_len=CTX_LEN, batch_size=BATCH_SIZE, shuffle=True)
with run:
    while token_count < TRAIN_TOKENS:
        X, y = get_batch(train_data, ctx_len=CTX_LEN, batch_size=BATCH_SIZE, shuffle=True)
        X = X.to(device)
        y = y.to(device)

        token_count = i * BATCH_SIZE * CTX_LEN

        preds = model(params=model_params, input_ids=X, vocab_size=len(chars))
        loss = torch.nn.functional.cross_entropy(input=preds, target=y[:, -1])
        total_loss += loss.item()
        loss.backward()

        with torch.no_grad():
            optim.step()
            optim.zero_grad()

            # for param in model_params.values():
            #     param -= LR * param.grad
            #     param.grad.zero_()

        if i % LOG_INTERVAL == 0:
            wandb.log(
                {
                    "loss": total_loss / LOG_INTERVAL,
                    "epoch": token_count // len(train_data),
                },
                step=token_count,
            )
            total_loss = 0

        if i % VALIDATION_INTERVAL == 0:
            j = 0
            for X_val, y_val in get_epoch(
                val_data, ctx_len=CTX_LEN, batch_size=4096, shuffle=False
            ):
                X_val = X_val.to(device)
                y_val = y_val.to(device)
                with torch.no_grad():
                    preds = model(
                        params=model_params, input_ids=X_val, vocab_size=len(chars)
                    )
                    loss = torch.nn.functional.cross_entropy(preds, y_val[:, -1])
                    val_total_loss += loss.item()
                    j += 1
            wandb.log({"val_loss": val_total_loss / j}, step=token_count, commit=True)

            prompts = get_batch(val_data, ctx_len=CTX_LEN, batch_size=5)[0].to(device)
            generated = generate(
                model=model,
                model_params=model_params,
                encoded_prompt=prompts,
                ctx_len=CTX_LEN,
                n_tokens=CTX_LEN * 2,
            )
            for p in generated:
                char_list = p.tolist()
                pre_prompt = char_list[:CTX_LEN]
                post_prompt = char_list[CTX_LEN:]

                print(f"{repr(decode(pre_prompt))}-> {repr(decode(post_prompt))}")

            val_total_loss = 0
        i += 1

'd higher and'-> ' said, "I want to play w'
'the tree, go'-> ' he was and the bird fle'
' wanted to e'-> 'at the bird and the bird'
's so happy h'-> 'er mom and dad. The bird'
'very day. Li'-> 'ly was so happy and said'
's a huge bal'-> 'l and the big store and '
'iny and had '-> 'a friends and said, "You'
'rry did not '-> 'like the story and said,'
' who never g'-> 'ood and said, "You are y'
'ple told him'-> ' that he was so happy to'
'mom was hurt'-> ' and said, "Thank you, M'
'd followed A'-> 'nna and saw a big tree w'
' found a big'-> ' tree with a big surpris'
'r dress?"\nSa'-> 'rah was so happy and the'
'he tall buil'-> 'd and said, "Thank you, '
'rt. They cri'-> 'ed to play with her frie'
'ould talk to'-> ' his friend and said, "I'
'ee. The bird'-> ' was so happy to have a '
'he chance to'-> ' help her friend the bir'
' bag said, "'-> 'I am sorry, we can came '


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


epoch,▁▁▁▁▁▁▁▁▁
loss,█▃▂▂▂▁▁▁▁
val_loss,█▄▂▁
epoch,0
loss,1.01236
val_loss,1.05973


In [40]:
torch.cuda.empty_cache()

In [41]:
prompts = get_batch(val_data, ctx_len=CTX_LEN, batch_size=5)[0].to(device)

In [86]:
generated = generate(
    model=model,
    model_params=model_params,
    encoded_prompt=prompts,
    ctx_len=CTX_LEN,
    n_tokens=300,
    temperature=1,
    top_k=50,
)
for p in generated:
    char_list = p.tolist()
    pre_prompt = char_list[:CTX_LEN]
    post_prompt = char_list[CTX_LEN:]

    print(f"{repr(decode(pre_prompt))}-> {repr(decode(post_prompt))}")

'er! \nThe nex'-> "t day, Mian's mommy was excited the juice. They always. One day, who had a special bying. They had lottle joge again. The smy loved the boy came. After to the hole around again.\nOnce upon a time, heavy, Timmy saw tight. A fincy only the world and lishes. She knew it to all her longerded to open the "
'yons and sta'-> 'y of a happy\'s toys asked toys to their can. From that day on, every dog! The cat was very sad. Her mom said, happy tow excited for a wards selfush an inside to be needet. Lily went to help other look promise toy cursy for is.\nOne day, she would little crreful, "We ble crying. They saw that the pine'
'hing unexpec'-> 'ted. It is tho lottle boy. He was excited to see wear appined to see Pally learn and went to the pandow. This yag cats. She glapped her laughed. She wolded Sue. Sue said, "No, so not having to marccold not started Tom and his differents the speeper- and they all their stickets and them a pudding on '
'ded to keep '-> 'behieset and nover

In [ ]:
import json

d = json.load(open("tmp/data00.json", "r"))

{'story': "Once upon a time, there was a cute puppy named Max. Max was very adorable with his big, brown eyes and wagging tail. One day, Max's owner, Emily, told him that they needed to go to the post office to mail a letter. Max didn't know what that meant, but he was excited to go for a car ride.\nAt the post office, Emily gave the letter to the nice lady behind the desk. The lady asked Emily for a number and Emily gave her one. Max didn't know what a number was, but he saw the lady type something on the computer.\nAfter they mailed the letter, Emily and Max went back to the car. Max was happy that they went on an adventure and he couldn't wait for the next one.",
 'instruction': {'prompt:': 'Write a short story (3-5 paragraphs) which only uses very simple words that a 3 year old child would understand. In the story, try to at some point use the verb "mail", the noun "number" and the adjective "adorable". Remember to only use simple words!',
  'words': ['mail', 'number', 'adorable'],